In [71]:
import numpy as np
import pickle as pkl
import pandas as pd

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pkl.dump(obj, f, pkl.HIGHEST_PROTOCOL)


def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pkl.load(f)

In [11]:
        products_cat = load_obj('../data/products_table')
        traffic_cat = load_obj('../data/traffic_table')
        purchased_cat = load_obj('../data/purchased_table')
        question_text_df = load_obj('../data/question_text_df')
        answer_text_df = load_obj('../data/answer_text')

In [20]:
def question_id_to_text(question, question_df):
    try:
        question_text = question_df.loc[question_df["PropertyDefinitionId"] == str(int(question)), "PropertyDefinition"].values[0]
    except IndexError:
        question_text = 'No text equivalent for question'
    return question_text

In [23]:
question_text = question_id_to_text('7302', question_text_df)
print(question_text)

Betriebssystem


In [25]:
traffic_cat["answers_selected"]

0         {'1': [0.39115999999999995], '7302': ['5767']}
1                           {'1': [0.39115999999999995]}
2                   {'7302': ['185245'], '11297': [0.0]}
3      {'746': ['278771', '306565', '365632'], '6885'...
4                                  {'16667': ['310362']}
5                                  {'11550': ['311476']}
6      {'12129': ['185639'], '1': [0.35559999999999997]}
7      {'12129': ['5306', '185639', '12936'], '1': [0...
8      {'12129': ['185639'], '7302': ['5767'], '11297...
9          {'11280': [1000000000.0], '6885': ['277226']}
10                                      {'1': [0.32004]}
11     {'16': ['294'], '746': ['306568', '365631', '2...
12     {'11297': [400000000000.0, 0.0], '6885': ['324...
13     {'9668': ['180435'], '7302': ['5767'], '11339'...
14                                      {'1': [0.42672]}
15                                      {'1': [0.42672]}
16             {'12129': ['185639'], '6885': ['277226']}
17                          {'1

In [16]:
traffic_cat["answers_selected"].iloc[3]

{'746': ['278771', '306565', '365632'], '6885': ['277226']}

Compute the list of all the filters used in history:

In [43]:
list_filters_used = []
i = 0
for t in traffic_cat["answers_selected"]:
    i += 1
    for k in t.keys():
        list_filters_used.append(k)

Number of different filters used:

In [44]:
unique_filters = set(list_filters_used)
len(unique_filters)

50

There are many filters that don't occur in product tables! I deleted them...

Names and frequency:

In [97]:
question_text_list = []
df_history = pd.DataFrame(columns=["ProductId", "text", "frequency"])
total_freq = 0
for f in unique_filters:
    question_text = question_id_to_text(f, question_text_df)
    if not question_text =='No text equivalent for question':
        question_text_list.append(question_text)
        freq = list_filters_used.count(f)
        total_freq += freq
        df_history.loc[len(df_history)] = [f, question_text, freq]
df_history["frequency"] = df_history["frequency"] / total_freq
df_history

,ProductId,text,frequency
0,16,Bildschirmoberfläche,0.0271536
1,697,Speicherkartentyp,0.00093633
2,13,Tastaturlayout,0.00093633
3,8745,Prozessortyp,0.0290262
4,460,Speicherkapazität,0.00280899
5,1,Bildschirmgrösse,0.250936
6,12129,Notebook Anwendungsbereich,0.114232
7,60,Datenspeicher Schnittstelle,0.00374532
8,748,Bildschirmtechnologie,0.00468165
9,7302,Betriebssystem,0.144195


In [98]:
def create_history(traffic_cat):
    #Compute the list of all the filters used in history
    list_filters_used = []
    i = 0
    for t in traffic_cat["answers_selected"]:
        i += 1
        for k in t.keys():
            list_filters_used.append(k)
    unique_filters = set(list_filters_used)
    question_text_list = []
    df_history = pd.DataFrame(columns=["ProductId", "text", "frequency"])
    total_freq = 0
    for f in unique_filters:
        question_text = question_id_to_text(f, question_text_df)
        if not question_text == 'No text equivalent for question':
            question_text_list.append(question_text)
            freq = list_filters_used.count(f)
            total_freq += freq
            df_history.loc[len(df_history)] = [f, question_text, freq]
    df_history["frequency"] = df_history["frequency"] / total_freq
    return df_history

In [103]:
df_history =create_history(traffic_cat)

In [130]:
df_history["ProductId"].values


array(['16', '697', '13', '8745', '460', '1', '12129', '60', '748',
       '7302', '11297', '746', '421', '11298', '11280', '11550', '11339',
       '7', '347', '4', '59', '9668', '6885', '182', '10526', '3195',
       '7202'], dtype=object)

In [151]:
alpha = 2
question_set = ["7202", "421", "2"]
Q_proba = np.zeros(len(question_set))
for i in range(len(question_set)):
    q_id = str(int(question_set[i]))
    Q_proba[i] = 1/len(question_set)
    if q_id in df_history["ProductId"].values:
        Q_proba[i] += alpha * df_history["frequency"].loc[df_history["ProductId"] == q_id].values[0]
Q_proba = Q_proba / Q_proba.sum()
print(Q_proba)

[0.33333333 0.33519553 0.33147114]


In [143]:
df_history["frequency"].loc[df_history["ProductId"] == str(int(MI_matrix[1,0]))].values[0]
#int(MI_matrix[0,0]) in df_history["ProductId"]

0.0018726591760299626

In [163]:
a = [1,2,3,2] 
a = set(a)
list(a)

[1, 2, 3]